In [1]:
# Cell 1: basic imports
import numpy as np
import pandas as pd
import datetime

In [2]:
# Cell 2: define store locations and IDs
store_info = pd.DataFrame({
    'store_id': range(1, 11),  # 10 stores
    'store_name': [f"Smith Toys Store {i}" for i in range(1, 11)],
    # Example cities (you can change or add more)
    'city': ['Austin', 'Boston', 'Chicago', 'Denver', 'Houston',
             'Los Angeles', 'Miami', 'New York', 'Seattle', 'Washington'],
    'state': ['TX', 'MA', 'IL', 'CO', 'TX', 'CA', 'FL', 'NY', 'WA', 'DC']
})

# Cell 3: define time period (e.g., 12 months)
start = datetime.date(2023, 1, 1)
end = datetime.date(2023, 12, 31)
months = pd.date_range(start, end, freq='MS').strftime('%Y-%m').tolist()

In [3]:
# Cell 4: define demographic categories
race_categories = [
    'White',
    'Black or African American',
    'American Indian or Alaska Native',
    'Asian',
    'Native Hawaiian or Other Pacific Islander',
    'Other',
    'Two or more races'
]

age_groups = ['0-12', '13-17', '18-34', '35-54', '55+']

In [4]:
# Cell 5: helper function to create random distribution that sums to 1
def random_distribution(n, seed=None):
    rng = np.random.default_rng(seed)
    vals = rng.random(n)
    return vals / vals.sum()

# Cell 6: build store demographic distributions
demographics = []
for idx, row in store_info.iterrows():
    # random seed for reproducibility; optional
    seed = int(row['store_id']) * 100
    
    race_dist = random_distribution(len(race_categories), seed=seed)
    age_dist = random_distribution(len(age_groups), seed=seed+1)
    
    # simulate median household income around different means per city
    base_income = rng_income = np.random.RandomState(seed+2).normal(60000, 15000)
    median_income = max(30000, base_income)  # floor
    
    demographics.append({
        'store_id': row['store_id'],
        'race_distribution': dict(zip(race_categories, race_dist)),
        'age_distribution': dict(zip(age_groups, age_dist)),
        'median_household_income': int(median_income)
    })

demographics_df = pd.DataFrame(demographics)

In [5]:
# Cell 7: generate monthly sales and profits
records = []
for _, store in store_info.iterrows():
    store_id = store['store_id']
    # pick a base average monthly sales for this store
    rng = np.random.default_rng(store_id * 50)
    base_sales = rng.normal(200000, 50000)  # average monthly sales
    base_sales = max(base_sales, 50000)     # floor
    
    for month in months:
        # add seasonality: higher in Nov-Dec, lower in Jan-Feb
        month_num = int(month.split('-')[1])
        season_factor = 1.0
        if month_num in [11, 12]:
            season_factor = 1.3
        elif month_num in [1, 2]:
            season_factor = 0.8
        
        sales = rng.normal(base_sales * season_factor, base_sales * 0.1)
        sales = max(0, sales)
        
        # profit margin: around 20-35%
        margin = rng.normal(0.27, 0.05)
        margin = np.clip(margin, 0.1, 0.5)
        
        profit = sales * margin
        
        # optional: number of transactions roughly correlated with sales
        avg_ticket = rng.normal(25, 5)
        avg_ticket = max(5, avg_ticket)
        transactions = int(sales / avg_ticket)
        
        records.append({
            'store_id': store_id,
            'month': month,
            'sales': round(sales, 2),
            'profit_margin': round(margin, 3),
            'profit': round(profit, 2),
            'transactions': transactions
        })

sales_df = pd.DataFrame(records)

In [6]:
# Cell 8: merge sales with store info
sales_full = sales_df.merge(store_info, on='store_id', how='left')

# Cell 9: optionally merge demographics
# For convenience, flatten race and age distributions into columns
race_flat = demographics_df['race_distribution'].apply(pd.Series)
race_flat.columns = [f"race_{c.replace(' ', '_').lower()}" for c in race_flat.columns]

age_flat = demographics_df['age_distribution'].apply(pd.Series)
age_flat.columns = [f"age_{c.replace('-', '_').replace('+','plus')}" for c in age_flat.columns]

demographics_flat = pd.concat(
    [demographics_df[['store_id', 'median_household_income']], race_flat, age_flat],
    axis=1
)

sales_full = sales_full.merge(demographics_flat, on='store_id', how='left')

# Cell 10: inspect
sales_full.head()

,store_id,month,sales,profit_margin,profit,transactions,store_name,city,state,median_household_income,...,race_american_indian_or_alaska_native,race_asian,race_native_hawaiian_or_other_pacific_islander,race_other,race_two_or_more_races,age_0_12,age_13_17,age_18_34,age_35_54,age_55plus
0,1,2023-01,168746.44,0.303,51208.97,9341,Smith Toys Store 1,Austin,TX,85021,...,0.070049,0.010416,0.23611,0.144643,0.191637,0.317328,0.12088,0.263945,0.198858,0.098988
1,1,2023-02,216824.31,0.260,56472.63,7835,Smith Toys Store 1,Austin,TX,85021,...,0.070049,0.010416,0.23611,0.144643,0.191637,0.317328,0.12088,0.263945,0.198858,0.098988
2,1,2023-03,241738.61,0.282,68227.70,9333,Smith Toys Store 1,Austin,TX,85021,...,0.070049,0.010416,0.23611,0.144643,0.191637,0.317328,0.12088,0.263945,0.198858,0.098988
3,1,2023-04,194224.77,0.261,50673.42,8713,Smith Toys Store 1,Austin,TX,85021,...,0.070049,0.010416,0.23611,0.144643,0.191637,0.317328,0.12088,0.263945,0.198858,0.098988
4,1,2023-05,224363.48,0.135,30291.21,13033,Smith Toys Store 1,Austin,TX,85021,...,0.070049,0.010416,0.23611,0.144643,0.191637,0.317328,0.12088,0.263945,0.198858,0.098988


In [7]:
# Cell 11: save dataset
sales_full.to_csv('smith_toys_sales_demo.csv', index=False)

In [8]:
# Cell 12
annual_sales = sales_full.groupby('store_id')['sales'].sum().reset_index()
annual_sales = annual_sales.merge(store_info, on='store_id')
annual_sales.sort_values('sales', ascending=False)

,store_id,sales,store_name,city,state
9,10,2957189.08,Smith Toys Store 10,Washington,DC
0,1,2647051.72,Smith Toys Store 1,Austin,TX
3,4,2607226.38,Smith Toys Store 4,Denver,CO
2,3,2501437.46,Smith Toys Store 3,Chicago,IL
6,7,2349412.87,Smith Toys Store 7,Miami,FL
5,6,2192122.41,Smith Toys Store 6,Los Angeles,CA
4,5,1923190.73,Smith Toys Store 5,Houston,TX
7,8,1814965.62,Smith Toys Store 8,New York,NY
1,2,1733299.72,Smith Toys Store 2,Boston,MA
8,9,1042866.03,Smith Toys Store 9,Seattle,WA


In [9]:
# Cell 13
avg_margin = sales_full.groupby('store_id')['profit_margin'].mean().reset_index()
avg_margin = avg_margin.merge(store_info, on='store_id')
avg_margin.sort_values('profit_margin', ascending=False)

,store_id,profit_margin,store_name,city,state
8,9,0.306750,Smith Toys Store 9,Seattle,WA
7,8,0.277667,Smith Toys Store 8,New York,NY
1,2,0.276917,Smith Toys Store 2,Boston,MA
4,5,0.274083,Smith Toys Store 5,Houston,TX
9,10,0.269917,Smith Toys Store 10,Washington,DC
6,7,0.269583,Smith Toys Store 7,Miami,FL
5,6,0.266417,Smith Toys Store 6,Los Angeles,CA
3,4,0.261500,Smith Toys Store 4,Denver,CO
0,1,0.259917,Smith Toys Store 1,Austin,TX
2,3,0.253833,Smith Toys Store 3,Chicago,IL


In [10]:
# Cell 14
corr = sales_full.groupby('store_id').agg({
    'sales': 'sum',
    'median_household_income': 'first'
}).corr()
print("Correlation between total sales and median income:\n", corr.loc['sales', 'median_household_income'])

Correlation between total sales and median income:
 -0.03980004961645319


In [11]:
# Cell 15: sales per transaction
sales_full['sales_per_transaction'] = sales_full['sales'] / sales_full['transactions']
sales_full[['store_id','month','sales_per_transaction']].head()

,store_id,month,sales_per_transaction
0,1,2023-01,18.065136
1,1,2023-02,27.673811
2,1,2023-03,25.901490
3,1,2023-04,22.291377
4,1,2023-05,17.215030
